# Computação Gráfica e Realidade Aumentada 
## &copy; 2025
<author>Authors: Guilherme Cabaço e Henrique Gomes</author>
<h1>Assignment 1</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

### Vertex e Fragment Shader

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

# A.1
### Cube class

In [ ]:
%%html

<script id="cube">
class cube extends CGRAobject{
    constructor(glcontext, color1, color2) {
        super(glcontext);
        var vertices = [
          -0.5,-0.5,-0.5,  //0
          -0.5,-0.5, 0.5,  //1
          -0.5, 0.5,-0.5,  //2
          -0.5, 0.5, 0.5,  //3
           0.5,-0.5,-0.5,  //4
           0.5,-0.5, 0.5,  //5
           0.5, 0.5,-0.5,  //6
           0.5, 0.5, 0.5   //7
        ];

        var indices = [
          1,5,7, 1,7,3,  // front
          0,2,6, 0,6,4,  // back
          0,1,3, 0,3,2,  // left
          4,6,7, 4,7,5,  // right
          2,3,7, 2,7,6,  // top
          0,4,5, 0,5,1   // bottom
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {
          const t = (vertices[i] + 1) / 2;
          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader, globalM = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();
        
        // Compute MVP
        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = globalM['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        // Bind vertex buffer → in_Position
        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        // Bind color buffer → in_Color
        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        // Bind index buffer
        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);

        // Draw indexed geometry
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>

### Sphere

In [ ]:
%%html

<script id="sphere">

class sphere extends CGRAobject {
    constructor(glcontext, color1, color2) {
        super(glcontext);
        this.gl = glcontext;

        const nStacks = 12;  // number of circles
        const nSlices = 12;  // number of vertices per circle
        const radius = 1.0;

        const vertices = [];
        const indices = [];
        const colors = [];

        // Top vertice
        vertices.push(0.0, radius, 0.0);
        colors.push(
            (color1[0] + color2[0]),
            (color1[1] + color2[1]),
            (color1[2] + color2[2])
        );
        const topIndex = 0;

        // Generate middle circles
        for (let i = 1; i < nStacks; i++) {
            const phi = Math.PI * i / nStacks; // between 0 and π
            const y = radius * Math.cos(phi);
            const r = radius * Math.sin(phi);
            
            for (let j = 0; j < nSlices; j++) {
                const theta = 2 * Math.PI * j / nSlices;
                const x = r * Math.cos(theta);
                const z = r * Math.sin(theta);
                
                vertices.push(x, y, z);
                
                //let t = j / (nSlices - 1);  // 0 → 1 around the circle
                
                const angle = j / (nSlices + 0.25) % 1; // 0 → 1 for full circle (0° → 360°)
                let t;
                if (angle < 0.5) {
                    // first 180°: color1 → color2
                    t = angle * 2; // goes 0 → 1
                } else {
                    // second 180°: color2 → color1
                    t = (1 - angle) * 2; // goes 1 → 0
                }
                
                const rC = color1[0] * (1 - t) + color2[0] * t;
                const gC = color1[1] * (1 - t) + color2[1] * t;
                const bC = color1[2] * (1 - t) + color2[2] * t;
                colors.push(rC, gC, bC);
            }
        }

        // Bottom vertice
        vertices.push(0.0, -radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const bottomIndex = vertices.length / 3 - 1;

        // Generate indices
        // Top cap
        for (let j = 0; j < nSlices; j++) {
            const next = (j + 1) % nSlices;
            indices.push(topIndex, 1 + j, 1 + next);
        }

        // Middle stacks
        for (let i = 0; i < nStacks - 2; i++) {
            const start = 1 + i * nSlices;
            const next = start + nSlices;
            
            for (let j = 0; j < nSlices; j++) {
                const k = (j + 1) % nSlices;
            
                indices.push(start + j, next + j, next + k);
                indices.push(start + j, next + k, start + k);
            }
        }

        // Bottom cap
        const lastRingStart = 1 + (nStacks - 2) * nSlices;
        for (let j = 0; j < nSlices; j++) {
            const next = (j + 1) % nSlices;
            indices.push(lastRingStart + j, bottomIndex, lastRingStart + next);
        }

        this.numIndices = indices.length;

        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);

        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.modelMat = glm.mat4(1.0);
    }

    draw(viewMat, projMat, shader, globalM = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = globalM['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}


</script>

### Cylinder

In [ ]:
%%html

<script id="cylinder">
class cylinder extends CGRAobject {
    constructor(glcontext, color1, color2) {
        super(glcontext);

        const numSegments = 12;
        const radius = 0.5;
        const height = 1.0;

        const vertices = [];
        const colors = [];
        const indices = [];

        // --- Create vertices ---
        // Bottom circle
        for (let i = 0; i < numSegments; i++) {
            const theta = (i / numSegments) * 2 * Math.PI;
            const x = radius * Math.cos(theta);
            const z = radius * Math.sin(theta);
            const y = -height / 2;
            vertices.push(x, y, z);

            const t = 0.3;
            colors.push(
                color1[0] * t + color2[0] * (1 - t),
                color1[1] * t + color2[1] * (1 - t),
                color1[2] * t + color2[2] * (1 - t)
            );
        }

        // Top circle
        for (let i = 0; i < numSegments; i++) {
            const theta = (i / numSegments) * 2 * Math.PI;
            const x = radius * Math.cos(theta);
            const z = radius * Math.sin(theta);
            const y = height / 2;
            vertices.push(x, y, z);

            const t = 0.7;
            colors.push(
                color1[0] * t + color2[0] * (1 - t),
                color1[1] * t + color2[1] * (1 - t),
                color1[2] * t + color2[2] * (1 - t)
            );
        }

        // --- Add center points for caps ---
        const bottomCenterIndex = vertices.length / 3;
        vertices.push(0, -height / 2, 0);
        let t = 0.2;
        colors.push(
            color1[0] * t + color2[0] * (1 - t),
            color1[1] * t + color2[1] * (1 - t),
            color1[2] * t + color2[2] * (1 - t)
        );

        const topCenterIndex = vertices.length / 3;
        vertices.push(0, height / 2, 0);
        t = 0.8;
        colors.push(
            color1[0] * t + color2[0] * (1 - t),
            color1[1] * t + color2[1] * (1 - t),
            color1[2] * t + color2[2] * (1 - t)
        );

        // --- Indices ---

        // Side faces (each quad → 2 triangles)
        for (let i = 0; i < numSegments; i++) {
            const next = (i + 1) % numSegments;
            const bottom1 = i;
            const bottom2 = next;
            const top1 = i + numSegments;
            const top2 = next + numSegments;

            // First triangle
            indices.push(bottom1, top1, bottom2);
            // Second triangle
            indices.push(bottom2, top1, top2);
        }

        // Bottom cap (fan)
        for (let i = 0; i < numSegments; i++) {
            const next = (i + 1) % numSegments;
            indices.push(bottomCenterIndex, next, i);
        }

        // Top cap (fan)
        for (let i = 0; i < numSegments; i++) {
            const next = (i + 1) % numSegments;
            indices.push(topCenterIndex, i + numSegments, next + numSegments);
        }

        // --- Buffers ---
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);

        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, globalM = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = globalM['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Cone

In [ ]:
%%html

<script id="cone">

class cone extends CGRAobject {
    constructor(glcontext, color1, color2) {
        super(glcontext);
        this.gl = glcontext;
        
        const n = 12;
        const radius = 1.0;
        const height = 2.0;
        const vertices = [];
        const indices = [];
        const colors = [];
        
        // top vertice
        vertices.push(0.0, height/2.0, 0.0);
        
        // base circle vertices
        for (let i = 0; i < n; i++) {
          const theta = (i / n) * 2 * Math.PI;
          const x = radius * Math.cos(theta);
          const z = radius * Math.sin(theta);
          vertices.push(x, -height/2, z);
        }
        
        // base center vertice
        vertices.push(0.0, -height/2.0, 0.0);
        
        const topIndex = 0;
        const baseCenterIndex = n + 1;
        
        // lateral indices
        for (let i = 1; i <= n; i++) {
          const next = (i % n) + 1;
          indices.push(topIndex, i, next);
        }
        
        // base indices
        for (let i = 1; i <= n; i++) {
          const next = (i % n) + 1;
          indices.push(baseCenterIndex, next, i);
        }

        // colors gradient
        for (let i = 0; i < vertices.length / 3; i++) {
          let t = 0.5; // default: middle blend

          if (i === 0 || i === n + 1) {
            // Top apex and base center: neutral middle color
            t = 0.5;
          } else {
            // Determine angular position around the circle (0 → 1)
            const angle = ((i - 1) / n) % 1;

            // Create wrap-around gradient
            if (angle < 0.5) {
              // first half (0° → 180°): color1 → color2
              t = angle * 2;
            } else {
              // second half (180° → 360°): color2 → color1
              t = (1 - angle) * 2;
            }
          }

          // Linear interpolation between color1 and color2
          const r = color1[0] * (1 - t) + color2[0] * t;
          const g = color1[1] * (1 - t) + color2[1] * t;
          const b = color1[2] * (1 - t) + color2[2] * t;

          colors.push(r, g, b);
        }

        
        this.numIndices = indices.length;
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);
    }
    
    draw(viewMat, projMat, shader, globalM = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = globalM['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }

}

</script>

### Disk

In [ ]:
%%html

<script id="disk">
class disk extends CGRAobject {
    constructor(glcontext, color1, color2) {
        super(glcontext);

        const numSegments = 12;
        const radius = 0.5;

        // --- Vertices ---
        // center first
        const vertices = [0.0, 0.0, 0.0];
        const colors = [];

        // center color (average)
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        // edge vertices around the circle
        for (let i = 0; i < numSegments; i++) {
            const theta = (i / numSegments) * 2 * Math.PI;
            const x = radius * Math.cos(theta);
            const y = radius * Math.sin(theta);
            const z = 0.0;
            vertices.push(x, y, z);

            // smooth color gradient along the rim
            const t = (i / numSegments);
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        // --- Indices (triangle fan) ---
        const indices = [];
        for (let i = 1; i <= numSegments; i++) {
            const next = (i % numSegments) + 1;
            indices.push(0, i, next);
        }

        // --- Buffers ---
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);

        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, globalM = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = globalM['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


In [1]:
%%html
<script id="locomotiveEngine">

class locomotiveEngine extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.cabine = new cube(this.gl, color1, color2);
        this.lower_body = new cube(this.gl, color1, color2);

        this.engine = new cylinder(this.gl, color1, color2);
        this.chimney = new cylinder(this.gl, color1, color2);
        this.nose = new cylinder(this.gl, color1, color2);
        this.cabine_roof = new cylinder(this.gl, color1, color2);

        this.chimney_top = new cone(this.gl, color1, color2);
        
        this.coolers = [];
        this.swheels = [];
        this.bwheels = [];
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.cabine.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.engine.setShader(shaderprog);
        this.chimney.setShader(shaderprog);
        this.nose.setShader(shaderprog);
        this.cabine_roof.setShader(shaderprog);
        this.chimney_top.setShader(shaderprog);
        
        for (let c of this.coolers) c.setShader(shaderprog);
        for (let w of this.swheels) w.setShader(shaderprog);
        for (let b of this.bwheels) b.setShader(shaderprog);
    }
    
    create_obj(){
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(6.0, 2.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // cabine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, 1.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.75, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine.setModelTransformation(transform);

        // engine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.25, 4.0, 2.25));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 1.0));
        transform = pos['*'](rot)['*'](scale);
        this.engine.setModelTransformation(transform);

        // chimney
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 1.0, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.9, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney.setModelTransformation(transform);

        // chimney top
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(0.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.2, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney_top.setModelTransformation(transform);

        // small wheels
        const smallWheelPositions = [
            glm.vec3(0.7, -0.25, 2.5),   // right front
            glm.vec3(0.7, -0.25, 1.25),   // right back
            glm.vec3(-0.7, -0.25, 2.5),  // left front
            glm.vec3(-0.7, -0.25, 1.25)   // left back
        ];

        this.swheels = [];

        for (let i = 0; i < smallWheelPositions.length; i++) {
            const isRightSide = smallWheelPositions[i].x > 0;
        
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            const wheel = new cylinder(this.gl, color1, color2);
            
            const rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
            const scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
            const pos = glm.translate(glm.mat4(1.0), smallWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.swheels.push(wheel);
        }

        // big wheel        
        const bigWheelPositions = [
            glm.vec3(-0.75, 0.15, -0.5),
            glm.vec3(-0.75, 0.15, -2.25),
            glm.vec3(0.75, 0.15, -0.5),
            glm.vec3(0.75, 0.15, -2.25)
        ];

        this.bwheels = [];

        for (let i = 0; i < bigWheelPositions.length; i++) {
            const isRightSide = bigWheelPositions[i].x > 0;
            
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            const wheel = new cylinder(this.gl, color1, color2);
            
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(1.75, 0.5, 1.75));
            const pos = glm.translate(glm.mat4(1.0), bigWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.bwheels.push(wheel);
        }

        // cabine roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.3, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine_roof.setModelTransformation(transform);

        // coolers
        const coolerPositions = [
            glm.vec3(0.6, 0.75, 1.75),
            glm.vec3(-0.6, 0.75, 1.75),
        ];

        this.coolers = [];

        for (let i = 0; i < coolerPositions.length; i++) {
            const cooler = new cylinder(this.gl, c1, c2);
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 2.5, 0.75));
            const pos = glm.translate(glm.mat4(1.0), coolerPositions[i]);
            const transform = pos['*'](rot)['*'](scale);
            cooler.setModelTransformation(transform);
            this.coolers.push(cooler);
        }

        // nose
        rot = glm.toMat4(glm.angleAxis(glm.radians(270.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 0.25, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 3.0));
        transform = pos['*'](rot)['*'](scale);
        this.nose.setModelTransformation(transform);
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        var localM = parentM ['*'](this.modelMat);
        
        this.lower_body.draw(viewM, projectionM, this.shader, localM);
        this.cabine.draw(viewM, projectionM, this.shader, localM);
        this.engine.draw(viewM, projectionM, this.shader, localM);
        this.chimney.draw(viewM, projectionM, this.shader, localM);
        this.chimney_top.draw(viewM, projectionM, this.shader, localM);
        this.nose.draw(viewM, projectionM, this.shader, localM);
        this.cabine_roof.draw(viewM, projectionM, this.shader, localM);
        
        for (let c of this.coolers) {
            c.draw(viewM, projectionM, this.shader, localM);
        }
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, localM);
        }
        
        for (let b of this.bwheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            b.setModelTransformation(b.baseTransform['*'](spin));
            b.draw(viewM, projectionM, this.shader, localM);
        }
    }
}


</script>

In [ ]:
%%html
<script id="locomotiveCarriage">

class locomotiveCarriage extends CGRAobject {
    constructor(glcontext, carriageSize = 2, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.carriageSize = carriageSize; // 1, 2, 3
        this.color1 = color1;
        this.color2 = color2;
        
        this.body = new cube(this.gl, color1, color2);
        this.lower_body = new cube(this.gl, color1, color2);

        this.roof = new cylinder(this.gl, color1, color2);

        this.swheels = [];
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.body.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.roof.setShader(shaderprog);
        
        for (let w of this.swheels) w.setShader(shaderprog);
    }
    
    create_obj(){
        const lengthMap = {1: 3.0, 2: 6.0, 3: 9.0};
        const bodyLength = lengthMap[this.carriageSize] || 6.0;
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 1.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // body
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 2.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.body.setModelTransformation(transform);

        
        this.swheels = [];
        let wheelZPositions;
        if(bodyLength === 3.0) wheelZPositions = [1.0, -1.0];
        else if(bodyLength === 6.0) wheelZPositions = [2.5, 1.25, -1.25, -2.5];
        else wheelZPositions = [3.75, 2.5, 1.25, -1.25, -2.5, -3.75];

        for(let z of wheelZPositions){
            for(let side of [-1,1]){ // left/right
                const color1 = side > 0 ? c1 : c2;
                const color2 = side > 0 ? c2 : c1;
                const wheel = new cylinder(this.gl, color1, color2);

                const rotW = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
                const scaleW = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
                const posW = glm.translate(glm.mat4(1.0), glm.vec3(0.7*side, -0.25, z));
                wheel.baseTransform = posW['*'](rotW)['*'](scaleW);

                this.swheels.push(wheel);
            }
        }

        // roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, bodyLength, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.roof.setModelTransformation(transform);

    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        var localM = parentM ['*'](this.modelMat);
        
        this.lower_body.draw(viewM, projectionM, this.shader, localM);
        this.body.draw(viewM, projectionM, this.shader, localM);
        this.roof.draw(viewM, projectionM, this.shader, localM);
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, localM);
        }
        
    }
}


</script>

In [ ]:
%%html
<script id="train">

class train extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.head = new locomotiveEngine(this.gl, color1, color2);
        this.c1 = new locomotiveCarriage(this.gl, 3, color1, color2);
        this.c2 = new locomotiveCarriage(this.gl, 1, color1, color2);
        this.c3 = new locomotiveCarriage(this.gl, 2, color1, color2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.head.setShader(shaderprog);
        this.c1.setShader(shaderprog);
        this.c2.setShader(shaderprog);
        this.c3.setShader(shaderprog);
    }
    
    create_obj(){
        this.headOffset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
        this.c1Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -8.5));
        this.c2Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -15.5));
        this.c3Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -21.0));
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        var localM = parentM['*'](this.modelMat);
        
        this.head.draw(viewM, projectionM, localM['*'](this.headOffset), counter);

        this.c1.draw(viewM, projectionM, localM['*'](this.c1Offset), counter);
        this.c2.draw(viewM, projectionM, localM['*'](this.c2Offset), counter);
        this.c3.draw(viewM, projectionM, localM['*'](this.c3Offset), counter);
    }
}


</script>

# Teste

In [ ]:
%%html
<canvas id="myCanvas1" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp1 extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        // Lets create cube object  
        this.cube1 = new cube(this.gl, [1,0,0], [0,0,1]);
        this.cube1.setShader(this.shaderprog);
        
        this.cube2 = new cylinder(this.gl, [1,0,0], [0,0,1]);
        this.cube2.setShader(this.shaderprog);
        
        this.cube3 = new disk(this.gl, [1,0,0], [0,0,1]);
        this.cube3.setShader(this.shaderprog);

        this.cone = new cone(this.gl, [1,0,0], [0,0,1]);
        this.cone.setShader(this.shaderprog); 
        
        this.sphere = new sphere(this.gl, [1,0,0], [0,0,1]);
        this.cone.setShader(this.shaderprog); 
        
        
        
        this.trans1 = glm.translate(glm.mat4(1.0),glm.vec3(5,6,7));
        this.trans2 = glm.translate(glm.mat4(1.0),glm.vec3(4,8,7));
        this.trans3 = glm.translate(glm.mat4(1.0),glm.vec3(5,8,10));
        this.trans4 = glm.translate(glm.mat4(1.0),glm.vec3(2,1,11));
        this.trans5 = glm.translate(glm.mat4(1.0),glm.vec3(6,8,13));

        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(12,13,15),glm.vec3(5,8,10),glm.vec3(0,1,0));
  
    }

    
    render(){
        
        this.shaderprog.startUsing();
        
        this.counter++;
        
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        var MVPlocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "MVP");
        
        
        var rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,0.0,1.0)));
        this.model1 = this.trans1['*'](rot);
        this.cube1.setModelTransformation(this.model1);
        
        this.cube1.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model2 = this.trans2['*'](rot);
        this.cube2.setModelTransformation(this.model2);
    
        this.cube2.draw(this.viewM, this.projectionM, this.shaderprog);
           
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,1.0,0.0)));
        this.model3 = this.trans3['*'](rot);
        this.cube3.setModelTransformation(this.model3);
        
        this.cube3.draw(this.viewM, this.projectionM, this.shaderprog);

        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model4 = this.trans4['*'](rot);
        this.cone.setModelTransformation(this.model4);
    
        this.cone.draw(this.viewM, this.projectionM, this.shaderprog);
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        this.model5 = this.trans5['*'](rot);
        this.sphere.setModelTransformation(this.model5);
    
        this.sphere.draw(this.viewM, this.projectionM, this.shaderprog);

          
    }
}

var app1 = new myapp1('myCanvas1');

app1.run();

</script>

### Teste objeto composto

In [ ]:
%%html
<canvas id="myCanvas4" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp4">

class objComp extends CGRAobject {
    constructor(glcontext){
        super(glcontext);
        this.cube1 = new cube(this.gl, [1,0,0], [0,0,1]);
        this.cube2 = new cube(this.gl, [1,0,0], [0,0,1]);
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.cube1.setShader(shaderprog);
        this.cube2.setShader(shaderprog);
    }
    
    create_obj(){ // put time to move everything
        var scale = glm.mat4(glm.mat3(1.1));
        this.cube1.setModelTransformation(glm.translate(glm.mat4(1.0), glm.vec3(0,-1,0)));
        this.cube2.setModelTransformation(glm.translate(glm.mat4(1.0), glm.vec3(0,1,0))['*'](scale));
    }
    
    drawit(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        //
        var localM = parentM ['*'](this.modelMat);
        this.cube1.drawit(viewM, projectionM, localM);
        this.cube2.drawit(viewM, projectionM, localM);
    }
}


class myapp4 extends DEECapp{    
    counter = 0;
    initialize(){
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.compObject = new locomotiveEngine(this.gl, [0.0, 0.75, 1.0], [1.0, 0.0, 0.25]);
        this.compObject.setShader(this.shaderprog);
        
        this.compObject2 = new locomotiveCarriage(this.gl, 3);
        this.compObject2.setShader(this.shaderprog);
        
        this.compObject3 = new train(this.gl, [0.0, 0.75, 1.0], [1.0, 0.0, 0.25]);
        this.compObject3.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45), 1.0, 0.1, 1000.0);
        this.viewM = glm.lookAt(glm.vec3(20, 20, 20), glm.vec3(0, 0, 0), glm.vec3(0, 1, 0));
    }
    
    
    
    render(){
        this.counter++;
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        
        var rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter), glm.vec3(0.0, 1.0, 0.0)));
        var modelM = rot;
        this.compObject.setModelTransformation(modelM);
                                                      
        //this.compObject.draw(this.viewM, this.projectionM, modelM, this.counter);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter), glm.vec3(0.0, 1.0, 0.0)));
        modelM = rot;
        this.compObject2.setModelTransformation(modelM);
                                                      
        //this.compObject2.draw(this.viewM, this.projectionM, modelM, this.counter);
        
        
        rot = glm.toMat4(glm.angleAxis(glm.radians(this.counter), glm.vec3(0.0, 1.0, 0.0)));
        modelM = rot;
        this.compObject3.setModelTransformation(modelM);
                                                      
        this.compObject3.draw(this.viewM, this.projectionM, modelM, this.counter);
    
        
    }

}


var app4 = new myapp4('myCanvas4');

app4.run();

</script>